<a href="https://colab.research.google.com/github/Prashitraj/Algorithmic-Trading/blob/main/Equal_Weight_S%26P_500_index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install xlsxwriter

In [ ]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math


In [ ]:
stocks = pd.read_csv('/content/drive/MyDrive/StockAnalysis/sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


In [ ]:
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'

In [ ]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

{'symbol': 'AAPL', 'companyName': 'Apple Inc', 'primaryExchange': '  ACDE/KSNBLSG SAMALTGTOLA(QR)NEE', 'calculationPrice': 'tops', 'open': None, 'openTime': None, 'openSource': 'facifilo', 'close': None, 'closeTime': None, 'closeSource': 'ffciloia', 'high': None, 'highTime': None, 'highSource': None, 'low': None, 'lowTime': None, 'lowSource': None, 'latestPrice': 124.148, 'latestSource': 'IEX real time price', 'latestTime': '1:51:52 PM', 'latestUpdate': 1635004513213, 'latestVolume': None, 'iexRealtimePrice': 123.004, 'iexRealtimeSize': 118, 'iexLastUpdated': 1642036472920, 'delayedPrice': None, 'delayedPriceTime': None, 'oddLotDelayedPrice': None, 'oddLotDelayedPriceTime': None, 'extendedPrice': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPriceTime': None, 'previousClose': 121.94, 'previousVolume': 103033056, 'change': -0.586, 'changePercent': -0.00491, 'volume': None, 'iexMarketPercent': 0.021230649810980218, 'iexVolume': 1079446, 'avgTotalVolume': 124714141

In [ ]:
price = data['latestPrice']
mcap = data['marketCap']
print(mcap/1000000000000)

2.104471668253


In [ ]:
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number Of Shares to Buy']
final_dataframe = pd.DataFrame(columns=my_columns)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number Of Shares to Buy


In [ ]:
final_dataframe.append(pd.Series([symbol,price,mcap,'N/A'],index=my_columns),ignore_index=True)

,Ticker,Stock Price,Market Capitalization,Number Of Shares to Buy
0,AAPL,124.148,2104471668253,N/A


In [ ]:
final_dataframe = pd.DataFrame(columns=my_columns)
for stock in stocks['Ticker']:
    symbol = stock
    api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(pd.Series([stock,data['latestPrice'],data['marketCap'],'N/A'],index=my_columns),ignore_index=True)    

In [ ]:
final_dataframe

In [ ]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [ ]:
symbol_groups = list(chunks(stocks['Ticker'],100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    # print(symbol_strings[i])
final_dataframe = pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(pd.Series([symbol,data[symbol]['quote']['latestPrice'],data[symbol]['quote']['marketCap'],'N/A'],index=my_columns),ignore_index=True)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number Of Shares to Buy
0,A,124.780,38285479514,N/A
1,AAL,23.142,14901178454,N/A
2,AAP,189.238,12273442373,N/A
3,AAPL,123.490,2052268918979,N/A
4,ABBV,107.110,190013259973,N/A
...,...,...,...,...
500,YUM,112.360,32288644235,N/A
501,ZBH,161.470,34105011454,N/A
502,ZBRA,476.780,25544799481,N/A
503,ZION,57.210,9130119901,N/A


In [ ]:
portfolio_size = input('Enter your portfolio size:')

try:
    val = float(portfolio_size)
except ValueError:
    print('Please enter an integer')

Enter your portfolio size:1000000


In [ ]:
position_size = val/len(final_dataframe.index)

for i in range(0,len(final_dataframe.index)):
    final_dataframe.loc[i,'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i,'Stock Price'])
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number Of Shares to Buy,Number of Shares to Buy
0,A,124.780,38285479514,N/A,15.0
1,AAL,23.142,14901178454,N/A,85.0
2,AAP,189.238,12273442373,N/A,10.0
3,AAPL,123.490,2052268918979,N/A,16.0
4,ABBV,107.110,190013259973,N/A,18.0
...,...,...,...,...,...
500,YUM,112.360,32288644235,N/A,17.0
501,ZBH,161.470,34105011454,N/A,12.0
502,ZBRA,476.780,25544799481,N/A,4.0
503,ZION,57.210,9130119901,N/A,34.0


In [ ]:
writer = pd.ExcelWriter("/content/drive/MyDrive/StockAnalysis/")